In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [2]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 26
A = 56
mass_nucleon = 0.938273 
# mass_nucleus = 11.178
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

# read dataframe from csv file
def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    df["Veff"] = 0.0089
    
    df["Eeff"] = df["E0"] + df["Veff"]
    df["Ep"] = df["E0"] - df["nu"]
    df["Ep_eff"] = df["Ep"] + df["Veff"]

    # calculate normalized cross section:
    df['normalization'] = 1.0
    df['normError'] = 0.0
    df['system_err'] = 0.0
    # df_norm = pd.read_csv('Data/Fe56_Norm.csv')
    # for _, row in df_norm.iterrows():
    #     mask = pd.Series([True] * len(df))
    #     if pd.notna(row['E0']):
    #         mask &= df['E0'] == row['E0']
    #     if pd.notna(row['ThetaDeg']):
    #         mask &= df['ThetaDeg'] == row['ThetaDeg']
    #     if pd.notna(row['dataSet']):
    #         mask &= df['dataSet'] == row['dataSet']
    #     df.loc[mask, 'normalization'] = row['normalization']
    #     try:
    #         df.loc[mask, 'system_err'] = float(row['system_err'])
    #     except (ValueError, TypeError):
    #         df.loc[mask, 'system_err'] = np.nan
    # df.loc[df['dataSet'] == 'Meziani:1984is', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff']) / 0.15)
    # df.loc[df['dataSet'] == 'Altemus:1973', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff']) / 0.15)
    df['normCross'] = df['cross'] * df['normalization']
    df['normCrossError'] = (np.sqrt(df['error']**2 + ((df['system_err'] * df['cross'])**2))) * df['normalization']
    print(df.loc[df['normalization'] == 1, 'dataSet'].unique())
    
    # calculate the kinematic variables:
    df["ThetaRad"] = df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"] = (np.sin(df["ThetaRad"] / 2))**2
    df["cos2(T/2)"] = (np.cos(df["ThetaRad"] / 2))**2
    df["tan2(T/2)"] = (np.tan(df["ThetaRad"] / 2))**2

    df["nuel"] = df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus)
    df["Ex"] = df["nu"] - (df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus))

    df["R"] = 1.1 * (df["A"])**(1/3) + 0.86 / ((df["A"])**(1/3))

    df["F2foc"] = (df["Eeff"] / df["E0"])**2
    df["Q2"] = 4 * df["E0"] * (df["Ep"]) * df["sin2(T/2)"]
    df["Q2eff"] = 4 * df["Eeff"] * df["Ep_eff"] * df["sin2(T/2)"]
    df["qv2"] = df["nu"]**2 + df["Q2eff"]
    df["qv"] = np.sqrt(df["qv2"])

    df["W2"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2eff"]
    df["epsilon"] = 1 / (1 + 2 * (1 + (df["nu"]**2) / df["Q2eff"]) * df["tan2(T/2)"])

    df["gamma"] = alpha_fine * df["Ep_eff"] * (df["W2"] - mass_nucleon**2) / (( 4 * ((np.pi)**2) * df["Q2eff"] * mass_nucleon * df["E0"]) * (1 - df["epsilon"]))
    df["Sig_R"] = df["normCross"] / df["gamma"]
    df["D_sig_R"] = df["error"] / df["gamma"]
    df["Sig_mott"] = 4 * (alpha_fine**2) * (df["Ep"]**2) * df["cos2(T/2)"] / (df["Q2"]**2)
    df["Sig_mott_eff"] = df["Sig_mott"] * df["E0"] / df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"] = (df["qv2"]**2) / (4 * (alpha_fine**2) * (df["Ep_eff"]**2) * (df["cos2(T/2)"] + 2 * (df["qv2"] / df["Q2eff"]) * df["sin2(T/2)"]))
    df["Hcc"] = df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"] = df["H"] * df["normCross"]
    df["Hstar_error(nb)"] = df["H"] * df["normCrossError"]
    df["Hstar_Sig(GeV)"] = df["Hstar_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hstar_error(GeV)"] = df["Hstar_error(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_Sig(nb)"] = df["Hcc"] * df["normCross"]
    df["Hcc_error(nb)"] = df["Hcc"] * df["normCrossError"]
    df["Hcc_Sig(GeV)"] = df["Hcc_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_error(GeV)"] = df["Hcc_error(nb)"] / ((0.1973269**2) * 10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvbin_names, right=True)
    df["qvcenter"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvcenters, right=True)
    df['qvcenter'] = pd.to_numeric(df['qvcenter'])
    df["Q2bin"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2bin_names, right=True)
    df["Q2center"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2centers, right=True)
    df['Q2center'] = pd.to_numeric(df['Q2center'])
    df = df.dropna()
    # now every row of data has a bin (Q2/qv) label

    # # this is for bc_qv_ex and bc_q2_ex
    # df['ExA'] = np.where(df['Ex'] >= 0.04, df['Ex'], 0.04)
    # df['nuA'] = np.where(df['Ex'] >= 0.04, df['nu'], df['nu'] + 0.04 - df['Ex'])
    # df["Q2effA"] = 4 * df["Eeff"] * (df["Eeff"] - df['nuA']) * df["sin2(T/2)"]
    # df["qv2A"] = df["nuA"]**2 + df["Q2effA"]
    # df["epsilonA"] = 1 / (1 + 2 * (1 + (df["nuA"]**2) / df["Q2effA"]) * df["tan2(T/2)"])


    return df

In [3]:
# read csv file
df = prepare_df('Data/Fe56_Fake.csv')
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 2.7) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.3) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.6) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.9) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 4.3) & (df['W2'] < 0.8))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 0.9) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 1.1) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Sealock:1989nx") & (df['E0'] == 1.299) & (df['W2'] < 0.75))]
df

['Altemus:1973' 'Arrington:1995hs' 'Arrington:1998ps' 'Baran:1988tw'
 'Chen:1990kq' 'Day:1993md' 'E04001' 'Hotta:1984' 'Meziani:1984is'
 'Sealock:1989nx']


,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,Veff,Eeff,...,Hstar_Sig(GeV),Hstar_error(GeV),Hcc_Sig(nb),Hcc_error(nb),Hcc_Sig(GeV),Hcc_error(GeV),qvbin,qvcenter,Q2bin,Q2center
0,26,56,0.1503,90.0,0.00323,3.588,0.07176,Altemus:1973,0.0089,0.1592,...,0.002923,0.000058,1.014476e+03,2.028953e+01,0.002605,0.000052,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056
1,26,56,0.1503,90.0,0.00523,127.660,2.55320,Altemus:1973,0.0089,0.1592,...,0.104050,0.002081,3.611151e+04,7.222302e+02,0.092741,0.001855,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056
2,26,56,0.1503,90.0,0.00723,259081.264,5181.62528,Altemus:1973,0.0089,0.1592,...,211.312208,4.226244,7.333799e+07,1.466760e+06,188.346009,3.766920,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056
3,26,56,0.1503,90.0,0.00923,32121.700,642.43400,Altemus:1973,0.0089,0.1592,...,26.223701,0.524474,9.101195e+06,1.820239e+05,23.373611,0.467472,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056
4,26,56,0.1503,90.0,0.01123,55409.640,1108.19280,Altemus:1973,0.0089,0.1592,...,45.289377,0.905788,1.571813e+07,3.143625e+05,40.367158,0.807343,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31983,26,56,1.2990,37.5,0.76900,6206.824,124.13648,Sealock:1989nx,0.0089,1.3079,...,131.764429,2.635289,5.061043e+07,1.012209e+06,129.977269,2.599545,"[0.878,1.302]",0.991,"[0.206,0.322]",0.265
31984,26,56,1.2990,37.5,0.77100,6202.456,124.04912,Sealock:1989nx,0.0089,1.3079,...,132.928172,2.658563,5.105742e+07,1.021148e+06,131.125228,2.622505,"[0.878,1.302]",0.991,"[0.206,0.322]",0.265
31985,26,56,1.2990,37.5,0.77300,6193.564,123.87128,Sealock:1989nx,0.0089,1.3079,...,134.006865,2.680137,5.147174e+07,1.029435e+06,132.189290,2.643786,"[0.878,1.302]",0.991,"[0.206,0.322]",0.265
31986,26,56,1.2990,37.5,0.77500,6184.256,123.68512,Sealock:1989nx,0.0089,1.3079,...,135.087590,2.701752,5.188684e+07,1.037737e+06,133.255357,2.665107,"[0.878,1.302]",0.991,"[0.206,0.322]",0.265


In [4]:
response_columns = ['i','RTTOT','RLTOT','RTQE','RLQE','RTNS','RLNS']
C12_factor = Z / 6
# 
QE_shift = 0
df = df[df['nu'] > QE_shift].reset_index(drop=True)
df_shift = df.copy()
df_shift['nu'] = df['nu'] - QE_shift
df_shift['Ex'] = df['Ex'] - QE_shift
df_shift['W2'] = df['W2'] - 2 * mass_nucleon * QE_shift
#

# calculate bc_qv_nu
df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE']
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor # qvc: qv at bin center
df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor # qvd: qv of data

df[['qv','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvd_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_qvd_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT qv_nu done.')

# calculate bc_qv_ex
df[['qvcenter','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','Ex']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor

df[['qv','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','Ex']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT qv_ex done.')

# calculate bc_qv_w2
df[['qvcenter','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT qv_w2 done.')

# calculate bc_q2_nu
df[['Q2center','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2c_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2c_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor

df[['Q2eff','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2d_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2d_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT q2_nu done.')

# calculate bc_q2_ex
df[['Q2center','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','Ex']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor

df[['Q2eff','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','Ex']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT q2_ex done.')

# calculate bc_q2_w2
df[['Q2center','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE']  
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] 
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE'] 
#
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT']) * C12_factor
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT']) * C12_factor
print('RL RT q2_w2 done.')


<>:17: SyntaxWarning: invalid escape sequence '\s'
<>:23: SyntaxWarning: invalid escape sequence '\s'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:42: SyntaxWarning: invalid escape sequence '\s'
<>:57: SyntaxWarning: invalid escape sequence '\s'
<>:63: SyntaxWarning: invalid escape sequence '\s'
<>:76: SyntaxWarning: invalid escape sequence '\s'
<>:82: SyntaxWarning: invalid escape sequence '\s'
<>:97: SyntaxWarning: invalid escape sequence '\s'
<>:103: SyntaxWarning: invalid escape sequence '\s'
<>:116: SyntaxWarning: invalid escape sequence '\s'
<>:122: SyntaxWarning: invalid escape sequence '\s'
<>:137: SyntaxWarning: invalid escape sequence '\s'
<>:143: SyntaxWarning: invalid escape sequence '\s'
<>:156: SyntaxWarning: invalid escape sequence '\s'
<>:162: SyntaxWarning: invalid escape sequence '\s'
<>:177: SyntaxWarning: invalid escape sequence '\s'
<>:183: SyntaxWarning: invalid escape sequence '\s'
<>:196: SyntaxWarning: invalid escape sequence '\s'
<>:202: SyntaxWarning

RL RT qv_nu done.
RL RT qv_ex done.
RL RT qv_w2 done.
RL RT q2_nu done.
RL RT q2_ex done.
RL RT q2_w2 done.


In [5]:
# determine bin-centering corrections
df["bc_qv_nu"] = 1.0
df["bc_qv_ex"] = 1.0
df["bc_qv_w2"] = 1.0
df["bc_q2_nu"] = 1.0
df["bc_q2_ex"] = 1.0
df["bc_q2_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN') and row['Ex'] >= 0.025:
            try:
                df.loc[index, 'bc_qv_nu'] = (row['epsilon'] * row['RL_qvc_nu'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_nu']) / (row['epsilon'] * row['RL_qvd_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_ex'] = (row['epsilon'] * row['RL_qvc_ex'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_ex']) / (row['epsilon'] * row['RL_qvd_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_w2 due to zero denominator.')

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]
    for index, row in picked.iterrows():
        if row['Q2center'] != float('NaN') and row['Ex'] >= 0.025:
            nu = (row['W2'] + Q2center - mass_nucleon**2) / (2 * mass_nucleon)
            qv2center = Q2center + nu**2
            try:
                df.loc[index, 'bc_q2_nu'] = (row['epsilon'] * row['RL_q2c_nu'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_nu']) / (row['epsilon'] * row['RL_q2d_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_ex'] = (row['epsilon'] * row['RL_q2c_ex'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_ex']) / (row['epsilon'] * row['RL_q2d_ex'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_w2'] = (row['epsilon'] * row['RL_q2c_w2'] + 0.5 * (qv2center / (Q2center)) * row['RT_q2c_w2']) / (row['epsilon'] * row['RL_q2d_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_w2 due to zero denominator.')

Skipping index 11464 for bc_qv_nu due to zero denominator.
Skipping index 11465 for bc_qv_nu due to zero denominator.
Skipping index 11466 for bc_qv_nu due to zero denominator.
Skipping index 11467 for bc_qv_nu due to zero denominator.
Skipping index 11468 for bc_qv_nu due to zero denominator.
Skipping index 11469 for bc_qv_nu due to zero denominator.
Skipping index 11470 for bc_qv_nu due to zero denominator.
Skipping index 27433 for bc_qv_nu due to zero denominator.
Skipping index 27434 for bc_qv_nu due to zero denominator.
Skipping index 27435 for bc_qv_nu due to zero denominator.
Skipping index 27436 for bc_qv_nu due to zero denominator.
Skipping index 27437 for bc_qv_nu due to zero denominator.
Skipping index 27438 for bc_qv_nu due to zero denominator.
Skipping index 27439 for bc_qv_nu due to zero denominator.
Skipping index 28140 for bc_qv_nu due to zero denominator.
Skipping index 28141 for bc_qv_nu due to zero denominator.
Skipping index 28142 for bc_qv_nu due to zero denominato

C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] 

Skipping index 28411 for bc_qv_nu due to zero denominator.
Skipping index 2130 for bc_qv_nu due to zero denominator.
Skipping index 2131 for bc_qv_nu due to zero denominator.
Skipping index 2132 for bc_qv_nu due to zero denominator.
Skipping index 2133 for bc_qv_nu due to zero denominator.
Skipping index 2134 for bc_qv_nu due to zero denominator.
Skipping index 2135 for bc_qv_nu due to zero denominator.
Skipping index 2136 for bc_qv_nu due to zero denominator.
Skipping index 2137 for bc_qv_nu due to zero denominator.
Skipping index 2138 for bc_qv_nu due to zero denominator.
Skipping index 2139 for bc_qv_nu due to zero denominator.
Skipping index 2140 for bc_qv_nu due to zero denominator.


C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] 

Skipping index 7273 for bc_qv_nu due to zero denominator.


C:\Users\Rhys\AppData\Local\Temp\ipykernel_17276\2304929384.py:21: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])


Skipping index 27433 for bc_q2_nu due to zero denominator.
Skipping index 27433 for bc_q2_w2 due to zero denominator.
Skipping index 27434 for bc_q2_nu due to zero denominator.
Skipping index 27434 for bc_q2_w2 due to zero denominator.
Skipping index 27435 for bc_q2_nu due to zero denominator.
Skipping index 27435 for bc_q2_w2 due to zero denominator.
Skipping index 27436 for bc_q2_nu due to zero denominator.
Skipping index 27436 for bc_q2_w2 due to zero denominator.
Skipping index 27437 for bc_q2_nu due to zero denominator.
Skipping index 27437 for bc_q2_w2 due to zero denominator.
Skipping index 27438 for bc_q2_nu due to zero denominator.
Skipping index 27438 for bc_q2_w2 due to zero denominator.
Skipping index 27439 for bc_q2_nu due to zero denominator.
Skipping index 27439 for bc_q2_w2 due to zero denominator.
Skipping index 28140 for bc_q2_nu due to zero denominator.
Skipping index 28140 for bc_q2_w2 due to zero denominator.
Skipping index 28141 for bc_q2_nu due to zero denominato

In [6]:
# save dataframe to csv
df.to_csv('Data/df_Fe56_Fake.csv',index=False)